In [ ]:
!pip install dash pandas
!pip install dash==2.0.0
!pip install -q jupyter_dash==0.3.0
!pip install -q dash-cytoscape
!pip install typing-extensions
!pip install pandas-datareader
!pip install pmdarima


In [ ]:
import base64
import datetime
import io

from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import dash
import plotly.express as px
import plotly.graph_objects as go
import plotly.tools as tls
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import to_datetime
from prophet import Prophet
from matplotlib import pyplot
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from datetime import datetime
from pandas import to_datetime
from prophet import Prophet
from matplotlib import pyplot

In [2]:
def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

def model_predict(df):
  train_size = 0.8
  split_idx = round(len(df)* train_size)
  train = df.iloc[:split_idx]
  test = df.iloc[split_idx:]
  
  #ARIMA
  df = df.iloc[1: , :]
  model = sm.tsa.arima.ARIMA(train,order=(1,0,1))
  model_fit = model.fit()
  modelvalues = model_fit.predict(start=1, end=len(df))
  df['Predicted_ARIMA'] = modelvalues.values
  predicted_arima = df['Predicted_ARIMA']

  #Auto ARIMA
  model = auto_arima(train, start_p=0, start_q=0)
  forecast = model.predict(n_periods=len(df))
  forecast = pd.DataFrame(forecast,index = df.index,columns=['Prediction'])
  df['Prediction'] = forecast.values

  #SARIMA
  model_SARIMA=SARIMAX(train,order=(2,1,0),seasonal_order=(0,1,0,12))
  model_SARIMA_fit=model_SARIMA.fit()
  modelvalues1 = model_SARIMA_fit.predict(start=1,end=len(df))
  df['Predicted_SARIMA']=modelvalues1.values
  predicted_sarima = df['Predicted_SARIMA']
  residuals=df['Close']-df['Predicted_SARIMA']

  #FB PROPHET
  df1 = pd.read_csv('LLOYDS_Monthly.csv')
  df1.drop(df1.columns.difference(['Date','Close']), 1, inplace=True)
  df1.columns = ['ds', 'y']
  df1['ds']= to_datetime(df1['ds'])

  train_size = 0.8
  split_idx = round(len(df1)* train_size)

  train1 = df1.iloc[:split_idx]
  test1 = df1.iloc[split_idx:]

  # define the model
  model = Prophet(daily_seasonality=True, weekly_seasonality=True)
  # fit the model
  model.fit(df1)
  forecast_3 = model.predict(test1)
  df_past = pd.DataFrame(forecast_3)
  # define the period for which we want a prediction
  future = list()
  for i in range(3,13):
      date = '2022-%02d' % i
      future.append([date])
  future = DataFrame(future)
  future.columns = ['ds']
  future['ds']= to_datetime(future['ds'])
  # use the model to make a forecast
  forecast_4 = model.predict(future)
  df_future = pd.DataFrame(forecast_4)
  # summarize the forecast
  #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

  return (df, df1, df_past)



app = dash.Dash()


app.layout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=False
    ),
    
    
    dcc.Graph(id='Mygraph')
])


def parse_data(contents, filename):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')), parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
            df.drop(df.columns.difference(['Date','Close']), 1, inplace=True)
            df.dropna(inplace = True)
            df,df1,df_past=model_predict(df)
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded), parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
            df.drop(df.columns.difference(['Date','Close']), 1, inplace=True)
            df.dropna(inplace = True)
            df, df1, df_past = model_predict(df)
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])
    return (df, df1, df_past)
        



@app.callback(Output('Mygraph', 'figure'),
              [Input('upload-data', 'contents'),
              Input('upload-data', 'filename')])

def update_graph(contents, filename):
    if contents:
      contents = contents[0]
      filename = filename[0]
      df, df1, df_past = parse_data(contents, filename)
      figure=go.Figure(data = [go.Scatter(
                                        x = df['Date'],
                                        y = df['Close'],
                                        mode = 'lines',
                                        name = 'Observed',
                                        line={'color': 'orange'}),
                              go.Scatter(
                                        x = df['Date'],
                                        y = df['Predicted_SARIMA'],
                                        mode = 'lines',
                                        name = 'Predicted',
                                        line={'color': 'green'}),
                               ],
                                    layout=go.Layout(title={'text':'Close prices Prediction for SARIMA model',
                                        'y':0.9,
                                        'x':0.5,
                                        'xanchor': 'center',
                                        'yanchor': 'top'},
                                        xaxis_title='Date', 
                                        yaxis_title='Closing price in pounds (£)')
                                    )
      return figure


app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
